In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math

In [5]:
interpreter = tf.lite.Interpreter(model_path='MovenetLightning.tflite')
interpreter.allocate_tensors()

In [6]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [7]:
EDGES = {
    (0, 1): 'm',  # Connects keypoints 0 (left shoulder) and 1 (right shoulder)
    (0, 2): 'c',  # Connects keypoints 0 (left shoulder) and 2 (left hip)
    (1, 3): 'm',  # Connects keypoints 1 (right shoulder) and 3 (right hip)
    (2, 4): 'c',  # Connects keypoints 2 (left hip) and 4 (left knee)
    (0, 5): 'm',  # Connects keypoints 0 (left shoulder) and 5 (left ankle)
    (0, 6): 'c',  # Connects keypoints 0 (left shoulder) and 6 (right hip)
    (5, 7): 'm',  # Connects keypoints 5 (left ankle) and 7 (left foot)
    (7, 9): 'm',  # Connects keypoints 7 (left foot) and 9 (left toe)
    (6, 8): 'c',  # Connects keypoints 6 (right hip) and 8 (right knee)
    (8, 10): 'c', # Connects keypoints 8 (right knee) and 10 (right ankle)
    (5, 6): 'y',  # Connects keypoints 5 (left ankle) and 6 (right hip)
    (5, 11): 'm', # Connects keypoints 5 (left ankle) and 11 (nose)
    (6, 12): 'c', # Connects keypoints 6 (right hip) and 12 (nose)
    (11, 12): 'y',# Connects keypoints 11 (nose) and 12 (nose)
    (11, 13): 'm',# Connects keypoints 11 (nose) and 13 (left eye)
    (13, 15): 'm',# Connects keypoints 13 (left eye) and 15 (left ear)
    (12, 14): 'c',# Connects keypoints 12 (nose) and 14 (right eye)
    (14, 16): 'c' # Connects keypoints 14 (right eye) and 16 (right ear)
}

HIGH_PLANK_EDGES = {
    (5, 11): 'm',   # Left shoulder to Left hip
    (11, 13): 'm',  # Left hip to Left knee
    (13, 15): 'm',  # Left knee to Left ankle
    (6, 12): 'c',   # Right shoulder to Right hip
    (12, 14): 'c',  # Right hip to Right knee
    (14, 16): 'c',  # Right knee to Right ankle
    (5, 7): 'm',    # Left shoulder to Left elbow
    (7, 9): 'm',    # Left elbow to Left wrist
    (6, 8): 'c',    # Right shoulder to Right elbow
    (8, 10): 'c'    # Right elbow to Right wrist
}



In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for edge, color in HIGH_PLANK_EDGES.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    draw_connections(frame, keypoints_with_scores, HIGH_PLANK_EDGES, 0.4)  # Using HIGH_PLANK_EDGES
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    cv2.imshow('MoveNet Lightning', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Check if 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows()
